In [1]:
import requests
import json
import tensorflow as tf
import base64

# --- HELPER UNTUK KONVERSI DATA ---
# Fungsi ini mengubah kamus Python biasa menjadi format tf.Example
def _create_tf_example(features):
    feature_map = {}
    for key, value in features.items():
        if isinstance(value, int):
            feature_map[key] = tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
        elif isinstance(value, float):
            feature_map[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
        # Anda bisa tambahkan elif untuk string jika ada
    return tf.train.Example(features=tf.train.Features(feature=feature_map))


# --- KODE UTAMA ---
# Data sampel yang akan dikirim
sample_data = {
    "battery_power": 842, "blue": 0, "clock_speed": 2.2, "dual_sim": 0,
    "fc": 1, "four_g": 0, "int_memory": 7, "m_dep": 0.6, "mobile_wt": 188,
    "n_cores": 2, "pc": 2, "px_height": 20, "px_width": 756, "ram": 2549,
    "sc_h": 9, "sc_w": 7, "talk_time": 19, "three_g": 0, "touch_screen": 0, "wifi": 1
}

# 1. Ubah data ke tf.Example
tf_example = _create_tf_example(sample_data)

# 2. Serialisasi (ubah menjadi string biner)
serialized_example = tf_example.SerializeToString()

# 3. Encode ke Base64 agar aman dikirim via JSON
encoded_example = base64.b64encode(serialized_example).decode('utf-8')

# 4. Buat payload dengan format yang benar
# Perhatikan formatnya: {"b64": "..."}
payload = {
  "instances": [
    {"b64": encoded_example}
  ]
}

# URL endpoint prediksi
url = "http://localhost:8501/v1/models/mobile-price-model:predict"

# Kirim request POST
response = requests.post(url, data=json.dumps(payload))
response_json = response.json()

# Sekarang, kunci 'predictions' seharusnya sudah ada!
predictions = response_json['predictions']
predicted_class_index = tf.argmax(predictions[0]).numpy()

print("Response JSON dari Server:", response_json)
print("\nHasil Prediksi (softmax):", predictions[0])
print("Prediksi Kelas Harga (0-3):", predicted_class_index)

2025-08-14 14:03:26.728048: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-14 14:03:26.754172: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-14 14:03:26.754206: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-14 14:03:26.755083: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-14 14:03:26.759627: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-14 14:03:26.760369: I tensorflow/core/platform/cpu_feature_guard.cc:1

Response JSON dari Server: {'predictions': [[0.0775459334, 0.779365778, 0.142757297, 0.000330977957]]}

Hasil Prediksi (softmax): [0.0775459334, 0.779365778, 0.142757297, 0.000330977957]
Prediksi Kelas Harga (0-3): 1
